In [1]:
# %matplotlib widget

In [2]:
# modified from test.ipynb and main.py

from matplotlib import pyplot as plt
from matplotlib import cm
import numpy as np
import GPy

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(level=logging.INFO)

from emukit.model_wrappers import GPyModelWrapper
from emukit.model_wrappers.gpy_quadrature_wrappers import BaseGaussianProcessGPy, RBFGPy

from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.loop import UserFunctionWrapper

from emukit.core import ParameterSpace, ContinuousParameter
from emukit.core.initial_designs import RandomDesign

from GPy.models import GPRegression

from skopt.benchmarks import branin as _branin
from emukit.test_functions import branin_function

from scse.api.simulation import run_simulation

from matplotlib.colors import LogNorm
from matplotlib import pyplot as plt

# Decision loops 
from emukit.experimental_design import ExperimentalDesignLoop
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.quadrature.loop import VanillaBayesianQuadratureLoop

# Acquisition functions 
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.experimental_design.acquisitions import ModelVariance
# from emukit.quadrature.acquisitions import IntegralVarianceReduction
from emukit.experimental_design.acquisitions import IntegratedVarianceReduction

# Acquistion optimizers
from emukit.core.optimization import GradientAcquisitionOptimizer

# Stopping conditions
from emukit.core.loop import FixedIterationsStoppingCondition
from emukit.core.loop import ConvergenceStoppingCondition

from emukit.bayesian_optimization.acquisitions.log_acquisition import LogAcquisition

In [3]:
# Libraries for multiprocessing
from multiprocess.pool import Pool
from tqdm import tqdm

In [4]:
from loop import *

## miniSCOT Functions

In [5]:
def invoke_miniscot(x):
    """
    Handling single API call to miniSCOT simulation given some inputs

    x contains parameter configs x = [x0 x1 ...]
    - The order of parameters in x should follow the order specified in the parameter_space declaration
    - E.g. here we specify num_batteries = x[0]
    """

    kwargs = {
        'time_horizon': 336,
        'num_batteries': int(x[0])
    }

    if len(x) == 2:
        kwargs.update({
            'max_battery_capacity': int(x[1])
        })
        
    cum_reward = run_simulation(**kwargs)
    
    return cum_reward[-1]

In [6]:
def f(X):
    """
    Handling multiple API calls to miniSCOT simulation given some inputs

    X is a matrix of parameters
    - Each row is a set of parameters
    - The order of parameters in the row should follow the order specified in the parameter_space declaration
    """
    Y = []
    for x in X:
        cum_reward = invoke_miniscot(x)

        # Note that we negate the reward; want to find min
        Y.append(-cum_reward[-1])

    Y = np.reshape(np.array(Y), (-1, 1))
    return Y

In [7]:
def f_multiprocess(X):
    """
    Handling multiple API calls to miniSCOT simulation given some inputs using multiprocessing.

    X is a matrix of parameters
    - Each row is a set of parameters
    - The order of parameters in the row should follow the order specified in the parameter_space declaration
    """
    
    # Set to None to use all available CPU
    max_pool = None
    with Pool(max_pool) as p:
        Y = list(
            tqdm(
                p.imap(invoke_miniscot, X),
                total=X.shape[0]
            )
        )

    # Note that we negate the reward; want to find min
    Y = -np.reshape(np.array(Y), (-1, 1))
    return Y

## Plotting Functions

In [8]:
def plot_reward(X, Y, labels):
    """
    Plots reward against a maximum of two dimensions.
    """

    plt.style.use('seaborn')
    fig = plt.figure(figsize=(10,10))

    order = np.argsort(X[:,0])
    
    if X.shape[1] == 1:
        ax = plt.axes()
        ax.plot(X[order,0], Y[order])
        ax.set_xlabel(labels[0])
        ax.set_ylabel("Cumulative reward")
    elif X.shape[1] == 2:
        ax = plt.axes(projection='3d')
        im = ax.plot_trisurf(X[order,0].flatten(), X[order,1].flatten(), Y[order].flatten(), cmap=cm.get_cmap('viridis'))
        fig.colorbar(im)
        ax.set_xlabel(labels[0])
        ax.set_ylabel(labels[1])
        ax.set_zlabel("Cumulative reward")
    else:
        raise ValueError('X has too many dimensions to plot - max 2 allowed')

    return fig, ax

## Investigation Parameter Space

In [9]:
max_num_batteries = 800
min_num_batteries = 200
min_battery_capacity = 140
max_battery_capacity = 160
num_data_points = 10

timsteps_per_week = 336
num_weeks = 1

In [10]:
num_batteries = DiscreteParameter('num_batteries', range(min_num_batteries, max_num_batteries+1))
max_battery_capacities = DiscreteParameter('max_battery_capacity', range(min_battery_capacity, max_battery_capacity+1))
time_horizon = DiscreteParameter('time_horizon', [i for i in range(0, num_weeks*timsteps_per_week, timsteps_per_week)])

use_1d = True
if use_1d: 
    parameter_space = ParameterSpace([num_batteries])
else: 
    parameter_space = ParameterSpace([num_batteries, max_battery_capacities])

design = RandomDesign(parameter_space)

X = design.get_samples(num_data_points)
X

array([[745],
       [281],
       [330],
       [588],
       [332],
       [790],
       [697],
       [205],
       [456],
       [644]])

## Example Run

The same code appears at the top of the Emukit cell below. Optionally run this to check whether we get a convex function.

In [ ]:
Y = f_multiprocess(X)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
plot_reward(X, Y, parameter_space.parameter_names)

In [ ]:
# just to temporarily break
stopping_cell

## Emukit Bayesian Optimisation

In [ ]:
successful_sample = False
num_tries = 0
max_num_tries = 3

use_default= False
use_ard=False

while not successful_sample and num_tries < max_num_tries: 
    
    print(f"CURRENT ATTEMPT #{num_tries}")
    
    X = design.get_samples(num_data_points)
    Y = f_multiprocess(X)
    
    # plot init values
    plot_reward(X, Y, parameter_space.parameter_names)
    
    # emulator model

    if use_default: 
        gpy_model = GPRegression(X, Y)
    else: 
        kernel = GPy.kern.RBF(1, lengthscale=1e1, variance=1e4, ARD=use_ard)
        gpy_model = GPy.models.GPRegression(X, Y, kernel, noise_var=1e-10)
    
    try: 
        gpy_model.optimize()
        print("okay to optimize")
        model_emukit = GPyModelWrapper(gpy_model)

        # Load core elements for Bayesian optimization
        expected_improvement = ExpectedImprovement(model=model_emukit)
        optimizer = GradientAcquisitionOptimizer(space=parameter_space)

        # Create the Bayesian optimization object
        batch_size = 3
        bayesopt_loop = BayesianOptimizationLoop(model=model_emukit,
                                                 space=parameter_space,
                                                 acquisition=expected_improvement,
                                                 batch_size=batch_size)

        # Run the loop and extract the optimum;  we either complete 10 steps or converge
        max_iters = 10
        stopping_condition = (
            FixedIterationsStoppingCondition(i_max=max_iters) | ConvergenceStoppingCondition(eps=0.01)
        )

        bayesopt_loop.run_loop(f_multiprocess, stopping_condition)
        print("successfully ran loop")
        successful_sample = True
        
    except: 
        num_tries += 1 


In [ ]:
num_tries

In [ ]:
# X, Y

In [ ]:
new_X, new_Y = bayesopt_loop.loop_state.X, bayesopt_loop.loop_state.Y

new_order = np.argsort(new_X[:,0])
new_X = new_X[new_order,:]
new_Y = new_Y[new_order]

# new_X, new_Y

## Visualize and Get Extrema

### Simple Plot

In [ ]:
plot_reward(new_X, new_Y, parameter_space.parameter_names)

### 2D Plot

In [ ]:
x_plot = np.reshape(np.array([i for i in range(0, max_num_batteries)]), (-1,1))
mu_plot, var_plot = model_emukit.predict(x_plot)

# plt.figure(figsize=(12, 8))
plt.figure(figsize=(7, 5))
LEGEND_SIZE = 15
plt.plot(new_X, new_Y, "ro", markersize=10, label="All observations")
plt.plot(X, Y, "bo", markersize=10, label="Initial observations")
# plt.plot(x_plot, y_plot, "k", label="Objective Function")
plt.plot(x_plot, mu_plot, "C0", label="Model")
plt.fill_between(x_plot[:, 0],
                 mu_plot[:, 0] + np.sqrt(var_plot)[:, 0],
                 mu_plot[:, 0] - np.sqrt(var_plot)[:, 0], color="C0", alpha=0.6)
plt.fill_between(x_plot[:, 0],
                 mu_plot[:, 0] + 2 * np.sqrt(var_plot)[:, 0],
                 mu_plot[:, 0] - 2 * np.sqrt(var_plot)[:, 0], color="C0", alpha=0.4)
plt.fill_between(x_plot[:, 0],
                 mu_plot[:, 0] + 3 * np.sqrt(var_plot)[:, 0],
                 mu_plot[:, 0] - 3 * np.sqrt(var_plot)[:, 0], color="C0", alpha=0.2)

plt.legend(loc=2, prop={'size': LEGEND_SIZE})
plt.xlabel(r"$x$")
plt.ylabel(r"$f(x)$")
plt.grid(True)
# plt.xlim(0, 25)
plt.show()

## 3D Plots

#### Inferred Surface

In [ ]:
plt.style.use('seaborn')
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')

im = ax.plot_trisurf(new_X[:,0].flatten(), new_X[:,1].flatten(), new_Y.flatten(), cmap=cm.get_cmap('viridis'), alpha=0.75)
fig.colorbar(im)

ax.scatter(X[:,0].flatten(), X[:,1].flatten(), Y.flatten(), s=100, marker="o", color="b", label="Initial observations")
ax.scatter(new_X[:,0].flatten(), new_X[:,1].flatten(), new_Y.flatten(), marker="x", color="r", label="All observations")

ax.legend(loc=2, prop={'size': LEGEND_SIZE})
ax.set_xlabel(r"$x_1$")
ax.set_ylabel(r"$x_2$")
ax.set_ylabel(r"$f(x)$")
ax.grid(True)

#### Prediction Surface

In [ ]:
mesh_X, mesh_Y = np.mgrid[1:max_num_batteries+1:1, min_battery_capacity:max_battery_capacity+1:1]
positions = np.vstack([mesh_X.ravel(), mesh_Y.ravel()]).T
mu_plot, var_plot = model_emukit.predict(positions)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
surf = ax.plot_surface(mesh_X, mesh_Y, mu_plot.reshape((500,21)), cmap=cm.coolwarm,linewidth=0, antialiased=False)

## Old Code

In [ ]:

# X = design.get_samples(num_data_points)
# Y = f(X)


In [ ]:
# # emulator model
# use_default= False
# use_ard=True
# if use_default: 
#     gpy_model = GPRegression(X, Y)
# else: 
#     kernel = GPy.kern.RBF(1, lengthscale=1e1, variance=1e4, ARD=use_ard)
#     gpy_model = GPy.models.GPRegression(X, Y, kernel, noise_var=1e-10)
# gpy_model.optimize()
# model_emukit = GPyModelWrapper(gpy_model)

In [ ]:
# # Load core elements for Bayesian optimization
# expected_improvement = ExpectedImprovement(model=model_emukit)
# optimizer = GradientAcquisitionOptimizer(space=parameter_space)

In [ ]:
# # Create the Bayesian optimization object
# batch_size = 3
# bayesopt_loop = BayesianOptimizationLoop(model=model_emukit,
#                                          space=parameter_space,
#                                          acquisition=expected_improvement,
#                                          batch_size=batch_size)


In [ ]:
# # Run the loop and extract the optimum;  we either complete 10 steps or converge
# max_iters = 10
# stopping_condition = FixedIterationsStoppingCondition(
#     i_max=max_iters) | ConvergenceStoppingCondition(eps=0.01)

# bayesopt_loop.run_loop(f, stopping_condition)